## Send multiple requests to a deployed Model

#### The sample code in this Notebook shows how a Python Client can be used to call a Model for getting Predictions for Insight

In [1]:
import os
from ibm_watson_machine_learning import APIClient

In [2]:
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
                   "token": token,
                   "instance_id" : "wml_local",
                   "url": os.environ['RUNTIME_ENV_APSX_URL'],
                   "version": "3.5"
}

In [3]:
wml_client = APIClient(wml_credentials)

### Set the ID of the deployment space.

In [4]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

### Get deployment space ID from the name
- <font color="red"> Please provide the name of the space you created</font>

In [5]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(wml_client, 'Prod_Deployment_Space')
print("Space UID = " + space_uid)

Space UID = 000ad43a-2c4e-41e9-aa3b-9a68e146b33b


### Set the deployment Space ID
Enter your Space UID below (output of the previous step). 

<i>Note: Expected output for the below command is 'SUCCESS'.</i>

In [6]:
wml_client.set.default_space("000ad43a-2c4e-41e9-aa3b-9a68e146b33b")  # Enter your Space UID here

'SUCCESS'

In [7]:
wml_client.deployments.list()

------------------------------------  -------------------------------------------------  -----  ------------------------
GUID                                  NAME                                               STATE  CREATED
8d2b005e-fe1a-44bc-9ddc-d1d459fdcc1d  WOS-INTERNAL-b115767b-d1df-4fe1-b207-882b6586b724  ready  2021-08-30T21:27:43.453Z
a6e68da2-d6cc-4f9a-a555-783b9e5f9440  Airline_Prod_Deployment                            ready  2021-08-30T18:10:55.021Z
------------------------------------  -------------------------------------------------  -----  ------------------------


### Identify the correct deployment ID from the above list, corresponding to the deployment to which you want to send the scoring request.

<i> Note: The deployment id can be found under the 'GUID' column. </i>

In [8]:
deployment_id = 'a6e68da2-d6cc-4f9a-a555-783b9e5f9440' # deploymentID

### Sending a single Request to Model for Scoring
Update the fields list to include the features used in the model, and the values list with corresponding sample values for each field.

In [9]:
fields = ["DAY","DAY_OF_WEEK","ORIGIN_AIRPORT","DESTINATION_AIRPORT","DEPARTURE_DELAY","TAXI_OUT","DISTANCE"]
values = [[23,5,"MSP","PHX",-2.0,13.0,983]]

In [10]:
payload_scoring = {"input_data": [{"fields": fields,"values": values}]}
payload_scoring
scoring_response = wml_client.deployments.score(deployment_id, payload_scoring)
#print(payload_scoring)
print(scoring_response)

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.989850401693257, 0.010149598306742965]]]}]}


### Sending multiple Requests to Model for Scoring

<b>Please note that you need to run the below cell multiple times, until you send close to 1000 requests. Send the requests in steps of 100 or less over a period of 1 hr.

For each numerical feature, use the training dataset to identify the range of values. For each categorical feature, use the training dataset to identify the unique values for that feature. 
</b>

In [22]:
import random
for i in range(0,100):
    DAY = random.randint(1,31)
    DAY_OF_WEEK = random.randint(1,7)
    ORIGIN_AIRPORT = random.choice(['MSP', 'RAP', 'STX', 'CLT', 'ATL', 'PHX', 'JFK', 'SDF', 'LAX',
       'DAL', 'TPA', 'LGA', 'ISP', 'PHL', 'DTW', 'JAX', 'BOS', 'EWR',
       'SAN', 'ORD', 'DEN', 'PDX', 'ATW', 'STL', 'LIT', 'BNA', 'HIB',
       'MSY', 'STT', 'SFO', 'CVG', 'SJU', 'ITO', 'BIL', 'MDW', 'BWI',
       'DFW', 'LAS', 'IAH', 'TXK', 'MIA', 'SNA', 'DCA', 'OMA', 'ABY',
       'COU', 'MCO', 'FAY', 'AEX', 'COS', 'ONT', 'SCC', 'OAK', 'SAV',
       'SEA', 'SLC', 'LFT', 'AUS', 'FLL', 'SMF', 'CID', 'ABQ', 'LEX',
       'SAT', 'IND', 'TVC', 'BTM', 'RDU', 'MSN', 'ASE', 'SUN', 'MGM',
       'BUF', 'PBI', 'TUL', 'FAR', 'HNL', 'TUS', 'SBN', 'GRR', 'ELP',
       'CHA', 'OGG', 'PSP', 'MKE', 'BZN', 'BHM', 'HPN', 'RKS', 'SGF',
       'MLB', 'PNS', 'BQN', 'OKC', 'IAD', 'CHS', 'AVL', 'BUR', 'ROA',
       'KOA', 'BMI', 'FLG', 'CLE', 'HRL', 'CAE', 'MCI', 'FAT', 'SJC',
       'BTR', 'HOU', 'LBB', 'PIT', 'XNA', 'LIH', 'RIC', 'CRP', 'IDA',
       'RNO', 'ERI', 'ABE', 'ANC', 'ORF', 'ILM', 'ELM', 'CMH', 'MLI',
       'CAK', 'DAY', 'DSM', 'SYR', 'TTN', 'LAN', 'ICT', 'MFE', 'ROC',
       'GCC', 'AMA', 'FSD', 'SBA', 'GEG', 'FWA', 'BLI', 'GSO', 'GTF',
       'PHF', 'GSP', 'FNT', 'VPS', 'BFL', 'PSG', 'PVD', 'GJT', 'GUC',
       'YAK', 'MEM', 'BOI', 'ALB', 'FSM', 'SHV', 'SRQ', 'SWF', 'MHT',
       'GTR', 'RSW', 'EKO', 'JAN', 'TLH', 'SCE', 'DRO', 'MRY', 'MYR',
       'ECP', 'SPI', 'GFK', 'CPR', 'BIS', 'GPT', 'WRG', 'GRI', 'JNU',
       'JAC', 'AGS', 'EYW', 'YUM', 'SGU', 'APN', 'MOB', 'PSC', 'HOB',
       'RDM', 'PIA', 'LGB', 'MTJ', 'CLL', 'RDD', 'CSG', 'MAF', 'OME',
       'HSV', 'TRI', 'SBP', 'SPS', 'BDL', 'HDN', 'LAR', 'STC', 'GGG',
       'EGE', 'FAI', 'GRB', 'GNV', 'JMS', 'MDT', 'MOT', 'LBE', 'CWA',
       'CLD', 'LSE', 'TWF', 'PIB', 'GRK', 'LAW', 'AZO', 'TYS', 'EAU',
       'ISN', 'ACY', 'ABI', 'BRO', 'MLU', 'ACV', 'DAB', 'MHK', 'LCH',
       'DHN', 'CRW', 'SAF', 'LRD', 'OAJ', 'LNK', 'PWM', 'MFR', 'SIT',
       'BET', 'TOL', 'TYR', 'ACT', 'BTV', 'ALO', 'CHO', 'PUB', 'RHI',
       'EWN', 'BPT', 'EVV', 'SMX', 'CNY', 'MSO', 'OTZ', 'MBS', 'LWS',
       'BJI', 'KTN', 'RST', 'BQK', 'ESC', 'PIH', 'EUG', 'FCA', 'CMX',
       'PBG', 'DLH', 'PSE', 'MKG', 'ROW', 'PLN', 'GCK', 'PAH', 'CDV',
       'SUX', 'CMI', 'VLD', 'DBQ', 'JLN', 'AVP', 'ABR', 'ILG', 'ADK',
       'SJT', 'INL'])
    DESTINATION_AIRPORT = random.choice(['PHX', 'SLC', 'MIA', 'ATL', 'BDL', 'TUL', 'MSY', 'TPA', 'DEN',
       'AUS', 'MDW', 'BWI', 'PBI', 'CLT', 'CIU', 'DFW', 'SJC', 'OKC',
       'FAT', 'ORD', 'IAH', 'MCO', 'MSP', 'SJU', 'SNA', 'MCI', 'GRB',
       'JFK', 'DAL', 'LAX', 'FLL', 'HNL', 'BOS', 'LAR', 'LGA', 'SAT',
       'TRI', 'MKE', 'IND', 'CMH', 'SGF', 'SAN', 'PIT', 'GRK', 'RDU',
       'ROC', 'GEG', 'OAK', 'TUS', 'STL', 'JAX', 'LAS', 'DTW', 'BRW',
       'CHS', 'DCA', 'AVL', 'MHT', 'CVG', 'SFO', 'HRL', 'RIC', 'EWR',
       'GRR', 'BQN', 'HPN', 'SBA', 'BPT', 'RAP', 'MLI', 'ITO', 'SEA',
       'BFL', 'BUR', 'HOU', 'ELP', 'MSN', 'PDX', 'PSC', 'LEX', 'PWM',
       'PVD', 'KOA', 'SBP', 'GSP', 'MOB', 'XNA', 'TYR', 'BNA', 'RSW',
       'CPR', 'SMF', 'CAK', 'BIS', 'PHL', 'OGG', 'BHM', 'RNO', 'ICT',
       'DSM', 'LIT', 'AVP', 'ATW', 'OMA', 'IAD', 'MDT', 'KTN', 'SYR',
       'AMA', 'LGB', 'RDD', 'MFR', 'SCC', 'PNS', 'PSP', 'ONT', 'PIA',
       'VPS', 'MGM', 'EKO', 'ISN', 'LBB', 'BUF', 'LIH', 'GNV', 'DIK',
       'HSV', 'CLE', 'ABQ', 'BMI', 'EUG', 'CEC', 'SMX', 'DRO', 'ORH',
       'GSO', 'ORF', 'CRW', 'MEM', 'EVV', 'SDF', 'CLL', 'ASE', 'TVC',
       'SGU', 'ALB', 'BTR', 'INL', 'SPI', 'LAN', 'MOT', 'WRG', 'CLD',
       'CDV', 'LRD', 'FAR', 'FLG', 'JAN', 'FSD', 'AGS', 'DAY', 'TYS',
       'STT', 'FNT', 'GFK', 'GJT', 'MFE', 'ROW', 'BIL', 'RKS', 'SRQ',
       'ILM', 'HIB', 'PSG', 'FAI', 'CDC', 'LAW', 'CID', 'MAF', 'CWA',
       'COU', 'ANC', 'MLU', 'PLN', 'CAE', 'BOI', 'ACV', 'AEX', 'JMS',
       'ECP', 'COS', 'FSM', 'IDA', 'JAC', 'ISP', 'GPT', 'EYW', 'ACY',
       'BTV', 'HLN', 'DLH', 'SAV', 'DVL', 'ABR', 'SHV', 'MHK', 'CRP',
       'SPS', 'RDM', 'SUX', 'MEI', 'JLN', 'GCC', 'HDN', 'LFT', 'ELM',
       'DHN', 'MYR', 'BZN', 'SUN', 'CHA', 'HOB', 'DAB', 'SCE', 'LWS',
       'EGE', 'MBS', 'ACT', 'CHO', 'MRY', 'JNU', 'ABI', 'PAH', 'LCH',
       'CSG', 'PSE', 'FWA', 'MMH', 'OTZ', 'DBQ', 'SIT', 'MKG', 'IMT',
       'SBN', 'TXK', 'RST', 'MSO', 'TWF', 'SJT', 'AZO', 'YUM', 'ERI',
       'FCA', 'ROA', 'OTH', 'ESC', 'MTJ', 'OAJ', 'ABE', 'TLH', 'PBG',
       'SAF', 'BRD', 'CMX', 'ABY', 'STC', 'BRO', 'COD', 'GUC', 'VLD',
       'LNK', 'IAG', 'STX', 'GTF', 'LBE', 'TTN', 'FAY', 'BLI', 'MLB',
       'PIH', 'GTR', 'ADQ', 'BTM', 'MQT', 'BET', 'BJI', 'CMI', 'BQK'])
    DEPARTURE_DELAY = random.randint(-35,1003)
    TAXI_OUT = random.randint(4,143)
    DISTANCE = random.randint(0,4983)
    values = [[DAY,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE]]
    payload_scoring = {"input_data": [{"fields": fields,"values": values}]}
    scoring_response = wml_client.deployments.score(deployment_id, payload_scoring)
    print(scoring_response)

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [3.285770915351094e-07, 0.9999996714229085]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [5.268749880826817e-10, 0.999999999473125]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.9995188158226298, 0.00048118417737017216]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [1.1445067116255814e-10, 0.9999999998855493]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [0.0, 1.0]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [1.66608948859448e-11, 0.9999999999833391]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [4.884981308350689e-15, 0.9999999999999951]]]}]}
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[1, [4.44089209850062